Python for Data Analysis, GMC, Vilnius University, 2025

# HW3: Exploratory Data Analysis (EDA)

- **Tasks in this homework are built around this file:** https://github.com/Tallivm/vu-python/raw/refs/heads/main/plantdata.npy. Download it beforehand and put into the same folder as your Notebook.
- You will need to import these packages: `numpy`, `pandas`, `matplotlib`, `urllib`, `json`. You are also allowed to import `tqdm` and `itertools`. Do not import any other packages.
- **You will need to upload your solutions into your Github repository** dedicated for the Python for Data Analysis course. Either make it public, or keep private but add Taisija as a collaborator (nickname: Tallivm).
- Write at least one custom function somewhere, but it is not required to put everything into functions. Do not write docstrings (function description comments).
- Keep prints informative.
- Do not create classes (you shouldn't know yet what are those).
- Do not change assert statements.

There are 5 tasks in this Notebook. They have slightly different numbers of points between them, with subpoints shown for each subtask e.g. (0.2p). You need to collect 8 points in total to get the maximum grade.

As previously, each task consists of a text cell with task description, a code cell to solve the task, and a code cell with `assert` statements to check your code for *some* possible errors.

Don't hesitate to contact me or Martynas if you are stuck.

In [ ]:
# your imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import urllib.request
import json

# 🔃 Task 1 (1 point): Load and examine the data

There is a lab called SNAILAB which studies various snails and their secretion as a cosmetic and pharmaceutical ingredient. SNAILAB researchers know that properties of snail secretion depend on what snails ate in last days. As snails aren't very fast animals, they tend to sit on plants they eat, meaning researchers can take plant samples directly under snails to research their diets.

The "plantdata.npy" file is a Numpy array containing some of those measurements.
- (0.1p) Load it into this Notebook as `raw_data`.
- (0.1p) Print out the shape and type of data.
- (0.4p) Calculate and save the total number of missing values as `n_missing`. Print out the ratio of total missing values in data.
- (0.4p) For each **numeric** data column, print out the minimum, average, median, and maximum values. Ignore missing values.

*Do not use Pandas for Task 1.*

In [ ]:
# your code
filename = "plantdata.npy"
try:
    raw_data = np.load(filename, allow_pickle=True)
    
    if not np.issubdtype(raw_data.dtype, np.number):
        raise ValueError("raw_data contains non-numeric columns")
        
    print(f"Data shape: {raw_data.shape}")
    print(f"Data type: {raw_data.dtype}")
    
    def count_missing_values(data):
        missing_mask = np.isnan(data)
        return np.sum(missing_mask)
    
    n_missing = count_missing_values(raw_data)
    total_elements = raw_data.size
    missing_ratio = n_missing / total_elements
    
    print(f"Ratio of missing values: {missing_ratio:.4f} ({missing_ratio*100:.2f}%)")
    
    print("\nColumn Statistics:")
    print("Column | Min | Average | Median | Max")
    print("-" * 50)
    
    for col in range(raw_data.shape[1]):
        column_data = raw_data[:, col]
        
        valid_data = column_data[~np.isnan(column_data)]
        
        if len(valid_data) > 0:
            min_val = np.min(valid_data)
            avg_val = np.mean(valid_data)
            median_val = np.median(valid_data)
            max_val = np.max(valid_data)
            
            print(f"{col:6} | {min_val:8.4f} | {avg_val:8.4f} | {median_val:8.4f} | {max_val:8.4f}")
        else:
            print(f"{col:6} | All values missing")
except FileNotFoundError:
    print(f"Warning: File '{filename}' not found in the current directory!")
    print("Please make sure the file is in your working folder.")
except Exception as e:
    print(f"Error loading file: {e}")

In [ ]:
assert type(raw_data) == np.ndarray
assert n_missing == 20873

# 👍 Task 2 (1 point): Make data more convenient to work with



Because the dataset is a 2D table, it should be much easier to work with it using Pandas.
- (0.1p) Create a DataFrame containing data from the Numpy array. Name it `raw_df`.
- (0.2p) Using `urllib`, load a data sheet for this dataset, which is a JSON file (https://raw.githubusercontent.com/Tallivm/vu-python/refs/heads/main/plant_datasheet.json). Name the result `datasheet`.
- (0.3p) Create a copy of `raw_df` named `named_df`. Use `datasheet` to correctly name `named_df` columns.
- (0.4p) Use the "mapping" keyword from `datasheet` to replace values in certain columns of `named_df` into strings. Don't write names of data columns directly in Notebook - make it automatic. Make sure `raw_df` stays unchanged.

In [ ]:
# your code
raw_df = pd.DataFrame(raw_data)
print("Created raw_df DataFrame")
url = "https://raw.githubusercontent.com/Tallivm/vu-python/refs/heads/main/plant_datasheet.json"
with urllib.request.urlopen(url) as response:
    datasheet = json.loads(response.read().decode('utf-8'))

named_df = raw_df.copy()
column_names = datasheet.get('column names', [])
if column_names:
    named_df.columns = column_names
else:
    print("Warning: No column names found in datasheet")

def apply_value_mappings(df, mapping_dict):
    for column_name, value_mapping in mapping_dict.items():
        if column_name in df.columns:
            reversed_mapping = {v: k for k, v in value_mapping.items()}
            df[column_name] = df[column_name].map(reversed_mapping)
            print(f"Applied mappings to column: {column_name}")
    return df

mappings = datasheet.get('mapping', {})
if mappings:
    named_df = apply_value_mappings(named_df, mappings)
    print("Applied all value mappings")
else:
    print("No mappings found in datasheet")

In [ ]:
assert type(raw_df) == pd.DataFrame
assert type(named_df) == pd.DataFrame
assert type(datasheet) == dict
assert raw_df.columns[0] == 0
assert named_df.columns[0] == 'plant_height'
assert named_df.loc[1, 'petal_color'] == 'red'

# 🔎 Task 3 (1 point): Detect problematic values

- (0.1p) Print out the number of missing values for each column in `named_df`.
- (0.1p) For each **categorical** column, print out its unique values.
- (0.3p) Plot and show a boxplot for each **numeric** column. It should be a single boxplot with all feature "boxes" side-by-side. Make sure all labels are readable and nothing overlaps on the plot.
- (0.5p) Using the IQR approach, count and print out the number of outliers in each **numeric** column. Save the total number of outliers into a variable `n_outliers_total`.
   - Any data point below $Q1 - 1.5 \text{IQR}$ or above $Q3 + 1.5 \text{IQR}$ can be considered an outlier.
   - $Q1$ is the 25% percentile, $Q3$ is the 75% percentile.
   - $\text{IQR} = Q3-Q1$.
   - Ignore missing values.


In [ ]:
# your code
print("Missing values per column:")
for col in named_df.columns:
    n_missing = named_df[col].isna().sum()
    print(f"{col}: {n_missing}")

print("\nUnique values in categorical columns:")
categorical_cols = ['leaf_shape_type', 'leaf_surface_type', 'petal_color']
for col in categorical_cols:
    unique_vals = named_df[col].unique()
    print(f"{col}: {list(unique_vals)}")

numeric_cols = [col for col in named_df.columns if col not in categorical_cols]
plt.figure(figsize=(12, 6))
named_df[numeric_cols].boxplot(showfliers=False)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

def count_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return len(outliers)

n_outliers_total = 0
for col in numeric_cols:
    n_outliers = count_outliers_iqr(named_df[col])
    n_outliers_total += n_outliers
    print(f"{col}: {n_outliers} outliers")

print(f"Total outliers: {n_outliers_total}")

In [ ]:
assert n_outliers_total == 3917

# 🧼 Task 4 (2 points): Clean the data

The `snail_age` column certainly looks strange. Some data cleaning may be needed. After sending the initial analysis to SNAILAB researchers, one of them sent you an email with some additional information:

________________

*Thanks for flagging those issues in the dataset. Let me clarify some things.*

*So the data came from several independent biologists working in different locations, and unfortunately they weren't all using the same units for snail age - some recorded it in years while others used days. You'll need to use a bit of common sense to figure out which is which, keeping in mind that some snails can live up to 10-15 years. For the cleaned dataset, we'd like everything standardized to years.*

*Regarding the missing values, it's a bit nuanced. In some cases, a missing value means someone simply forgot to take that measurement, and we want those samples removed entirely from the analysis. But many missing values are actually meaningful because that measurement was impossible to get - please replace those with an appropriate constant like 0 or "N/A".*

*One more thing about the body measurements: we measured body length from the tip of the snail's tail all the way to the tip of the right eye tentacle. Biologically speaking, the eye tentacles can't be longer than about 40% of the total body length, so if you're seeing values that exceed that, they're definitely measurement errors. We'd like you to replace those incorrect eye length values using 20% of the corresponding body length.*

________________

- (1.2p) According to this new information, get a new DataFrame called `clean_df` which **should not have any missing values** whatsoever. Keep `named_df` unchanged. For any removed or replaced missing values, add a comment with brief explanation why it was done.
- (0.3p) Repair the eye length column.
- (0.4p) Repair the snail age column. Plot two histograms for the snail age data, before and after, side by side. Increase the number of bins in plots for better resolution.
- (0.1p) Repeat the boxplot from Task 3 but with `clean_df`.

*Do not remove or replace outliers, leave them in the data.*

In [ ]:
# your code
clean_df = named_df.copy()

print("Handling missing values:")
initial_missing = clean_df.isna().sum().sum()
print(f"Initial total missing values: {initial_missing}")

#Stulpeliai kur tuščios reikšmės pašalinamos, nes tikėtina pamiršta pamatuoti.
cols_to_check_removal = ['plant_height', 'stem_bending', 'leaf_size', 'flower_radius', "snail_age", "snail_body_length"]
rows_before = len(clean_df)
clean_df = clean_df.dropna(subset=cols_to_check_removal)
rows_after = len(clean_df)
print(f"Removed {rows_before - rows_after} rows with missing basic plant measurements")

#Stulpeliai kur tuščios reikšmės pakeičiamos į tam tikrą default, nes NaN tikėtina turi biologinę reikšmę.
clean_df['n_petals'] = clean_df['n_petals'].fillna(0)
clean_df['petal_size'] = clean_df['petal_size'].fillna(0)
clean_df['petal_color'] = clean_df['petal_color'].fillna('N/A')
clean_df["snail_eye_length"] = clean_df["snail_eye_length"].fillna(0)
print("Replaced missing petal measurements with 0 or 'N/A' (no flowers present) and missing snail eye length measurements with 0 (indicates that eyes were not visible to the researcher)")

#should not have any missing values whatsoever
final_missing = clean_df.isna().sum().sum()
print(f"Final total missing values: {final_missing}")


print("\nRepairing eye length measurements:")
def repair_eye_length(row):
    body_length = row['snail_body_length']
    eye_length = row['snail_eye_length']
    
    if eye_length > 0.4 * body_length:
        return 0.2 * body_length
    return eye_length

clean_df['snail_eye_length'] = clean_df.apply(repair_eye_length, axis=1)
invalid_eyes = ((named_df['snail_eye_length'] > 0.4 * named_df['snail_body_length']) & 
                named_df['snail_eye_length'].notna()).sum()
print(f"Repaired {invalid_eyes} biologically impossible eye length measurements")


print("\nRepairing snail age column:")
def convert_to_years(age):
    if age > 20:
        return age / 365.25  #365.25 kad įvertinti leap years
    return age

clean_df['snail_age'] = clean_df['snail_age'].apply(convert_to_years)


plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(named_df['snail_age'].dropna(), bins=100, color='red')
plt.title('Snail Age Before Repair')
plt.xlabel('Age (mixed units)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(clean_df['snail_age'], bins=100, color='green')
plt.title('Snail Age After Repair (Years)')
plt.xlabel('Age (years)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

converted_ages = (named_df['snail_age'] > 20).sum()
print(f"Converted {converted_ages} age values from days to years")


print("\nNew boxplot for clean_df numeric columns:")
plt.figure(figsize=(12, 6))
clean_df[numeric_cols].boxplot(showfliers=False)
plt.xticks(rotation=45)
plt.title('Numeric Columns (Clean Data)')
plt.tight_layout()
plt.show()

In [ ]:
assert list(named_df.columns) == list(clean_df.columns)
assert len(named_df) > len(clean_df)

# 🖼️ Task 5 (3 points): Visualize the data

You were asked to create some beautiful EDA plots from `clean_df`. Create 3 plots in total. You will need to save each of them as a PDF image (filenames are provided), as well as output in the Notebook. Make sure everything is readable on the plot, no overlapping parts, cropped/unreadable texts, etc.

1. (1p) **Feature distributions** ("data_features.pdf")
   - A subplot grid (so, at least 2 rows or 2 columns), with as many subplots as there are features (columns) in the data.
   - For each feature, plot either histogram or barplot depending on whether the feature is numeric (continuous/ranked) or categorical. The determination of feature type should be automatic, without manually writing column names as strings anywhere.
   - For each numeric feature, add a red dashed vertical line indicating the average, and a green dashed vertical line for the median.
   - Each histogram should have a title (feature name).
   - Increase bin numbers for better resolution.
   - Color all histograms in a color of your choice.

2. (1p) **Feature correlations** ("data_correlation.pdf")
   - A heatmap showing Spearman's correlation coefficients between all **numeric** features (reminder: you cannot use `seaborn`).
   - The heatmap should have visible values (in [-100%, 100%] range, rounded), correct X and Y ticks (feature names), and a colorbar.
   - Use a diverging colormap and ensure that 0% correlation has the middle color (check `vmin` and `vmax` parameters).
   - The title of the plot should be "Spearman correlation between features".

3. (1p) **Scatterplots of least and most correlated features** ("data_relationships.pdf")
   - Two scatterplots side-by-side, first showing tho least correlated features, second - two most correlated.
   - Least and most correlated features should be found automatically from the correlation matrix calculated for the second plot. Autocorrelation doesn't  count. Keep in mind that -100% is a huge correlation, just negative.
   - Plots should have correctly named X and Y axes.
   - Plot titles should contain Spearman correlation coefficients.
   - Color dots according to the petal colors. Dot colors can be different from the color names used in data, but try to match them (except for the white color, obviously). Make dots smaller and more transparent than default ones.

In [ ]:
# your code

n_features = len(clean_df.columns)
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

plt.figure(figsize=(16, 4 * n_rows))

for i, col in enumerate(clean_df.columns, 1):
    plt.subplot(n_rows, n_cols, i)
    
    if clean_df[col].dtype in ['float64', 'int64']:
        plt.hist(clean_df[col], bins=100, color='skyblue', alpha=0.7)
        plt.axvline(clean_df[col].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        plt.axvline(clean_df[col].median(), color='green', linestyle='--', linewidth=2, label='Median')
    else:
        value_counts = clean_df[col].value_counts()
        plt.bar(value_counts.index, value_counts.values, color='lightcoral', alpha=0.7)
        plt.xticks(rotation=45)
    
    plt.title(col)
    if i == 1 and clean_df[col].dtype in ['float64', 'int64']:
        plt.legend()

plt.tight_layout()
plt.savefig('data_features.pdf', bbox_inches='tight')
plt.show()


numeric_cols_for_corr = clean_df.select_dtypes(include=['float64', 'int64']).columns
correlation_matrix = clean_df[numeric_cols_for_corr].corr(method='spearman')

plt.figure(figsize=(10, 8))
im = plt.imshow(correlation_matrix * 100, cmap='RdBu_r', vmin=-100, vmax=100)

for i in range(len(correlation_matrix)):
    for j in range(len(correlation_matrix)):
        plt.text(j, i, f'{correlation_matrix.iloc[i, j]*100:.0f}%', 
                ha='center', va='center', fontsize=8)

plt.xticks(range(len(numeric_cols_for_corr)), numeric_cols_for_corr, rotation=45)
plt.yticks(range(len(numeric_cols_for_corr)), numeric_cols_for_corr)
plt.colorbar(im, label='Correlation (%)')
plt.title('Spearman correlation between features')
plt.tight_layout()
plt.savefig('data_correlation.pdf', bbox_inches='tight')
plt.show()


corr_pairs = []
for i in range(len(correlation_matrix)):
    for j in range(i + 1, len(correlation_matrix)):
        corr_val = correlation_matrix.iloc[i, j]
        corr_pairs.append((numeric_cols_for_corr[i], numeric_cols_for_corr[j], corr_val))

corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[2]))

least_corr = corr_pairs_sorted[0] 
most_corr = corr_pairs_sorted[-1]  
color_map = {
    'white': 'lightgray',
    'red': 'red', 
    'yellow': 'yellow',
    'blue': 'blue',
    'purple': 'purple',
    'N/A': 'black'
}

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
for color_name, color_code in color_map.items():
    mask = clean_df['petal_color'] == color_name
    plt.scatter(clean_df.loc[mask, least_corr[0]], 
                clean_df.loc[mask, least_corr[1]],
                c=color_code, label=color_name, alpha=0.6, s=20)
plt.xlabel(least_corr[0])
plt.ylabel(least_corr[1])
plt.title(f'Least correlated: r = {least_corr[2]:.3f}')
plt.legend()

plt.subplot(1, 2, 2)
for color_name, color_code in color_map.items():
    mask = clean_df['petal_color'] == color_name
    plt.scatter(clean_df.loc[mask, most_corr[0]], 
                clean_df.loc[mask, most_corr[1]],
                c=color_code, label=color_name, alpha=0.6, s=20)
plt.xlabel(most_corr[0])
plt.ylabel(most_corr[1])
plt.title(f'Most correlated: r = {most_corr[2]:.3f}')
plt.legend()

plt.tight_layout()
plt.savefig('data_relationships.pdf', bbox_inches='tight')
plt.show()

print(f"Least correlated: {least_corr[0]} vs {least_corr[1]} (r = {least_corr[2]:.3f})")
print(f"Most correlated: {most_corr[0]} vs {most_corr[1]} (r = {most_corr[2]:.3f})")

In [ ]:
import os  # do not delete this import
assert os.path.exists("data_features.pdf")
assert os.path.exists("data_correlation.pdf")
assert os.path.exists("data_relationships.pdf")

*Disclaimer: there is no lab called SNAILAB, and `plantdata.npy` is purely synthetic. Do not use this analysis to learn about snails. The data does not contain any information about the immortal snail. Do not use this analysis to evade the immortal snail. Evading the immortal snail is futile.*